In the previous session we trained a model for predicting churn and evaluated it. Now let's deploy it

In [28]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

from sklearn.pipeline import make_pipeline

In [29]:
data = 'https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-03-churn-prediction/WA_Fn-UseC_-Telco-Customer-Churn.csv'

In [30]:
!wget $data -O data-week-3.csv

--2026-01-04 07:25:16--  https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-03-churn-prediction/WA_Fn-UseC_-Telco-Customer-Churn.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 977501 (955K) [text/plain]
Saving to: ‘data-week-3.csv’

data-week-3.csv     100%[===================>] 954.59K  --.-KB/s    in 0.05s   

2026-01-04 07:25:16 (17.1 MB/s) - ‘data-week-3.csv’ saved [977501/977501]



In [31]:
df = pd.read_csv('data-week-3.csv')

df.columns = df.columns.str.lower().str.replace(' ', '_')

categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)

for c in categorical_columns:
    df[c] = df[c].str.lower().str.replace(' ', '_')

df.totalcharges = pd.to_numeric(df.totalcharges, errors='coerce')
df.totalcharges = df.totalcharges.fillna(0)

df.churn = (df.churn == 'yes').astype(int)

In [32]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)

In [33]:
numerical = ['tenure', 'monthlycharges', 'totalcharges']

categorical = [
    'gender',
    'seniorcitizen',
    'partner',
    'dependents',
    'phoneservice',
    'multiplelines',
    'internetservice',
    'onlinesecurity',
    'onlinebackup',
    'deviceprotection',
    'techsupport',
    'streamingtv',
    'streamingmovies',
    'contract',
    'paperlessbilling',
    'paymentmethod',
]

In [34]:
pipeline = make_pipeline(
    DictVectorizer(),
    LogisticRegression(C=C, max_iter=1000, solver = 'liblinear')
)

In [35]:
def train(df_train, y_train, C=1.0):
    dicts = df_train[categorical + numerical].to_dict(orient='records')
    pipeline.fit(dicts, y_train)

    return pipeline

In [36]:
def predict(df, pipeline):
    dicts = df[categorical + numerical].to_dict(orient='records')
    y_pred = pipeline.predict_proba(dicts)[:, 1]

    return y_pred

In [37]:
C = 1.0
n_splits = 5

In [38]:
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=1)

scores = []

for train_idx, val_idx in kfold.split(df_full_train):
    df_train = df_full_train.iloc[train_idx]
    df_val = df_full_train.iloc[val_idx]

    y_train = df_train.churn.values
    y_val = df_val.churn.values

    pipeline  = train(df_train, y_train, C=C)
    y_pred = predict(df_val, pipeline )

    auc = roc_auc_score(y_val, y_pred)
    scores.append(auc)

print('C=%s %.3f +- %.3f' % (C, np.mean(scores), np.std(scores)))

C=1.0 0.841 +- 0.007


In [39]:
scores

[np.float64(0.8423240260300963),
 np.float64(0.8453247086478611),
 np.float64(0.8337066024483243),
 np.float64(0.8323627454115241),
 np.float64(0.8521736060995889)]

In [40]:
pipeline  = train(df_full_train, df_full_train.churn.values, C=1.0)
y_pred = predict(df_test, pipeline )

y_test = df_test.churn.values
auc = roc_auc_score(y_test, y_pred)
auc

np.float64(0.8579400803839363)

Save the model using Pickel

In [41]:
import pickle

In [42]:
output_file = f'model_C = {C}.bin'
output_file

'model_C = 1.0.bin'

In [43]:
#f_out = open(output_file, 'wb')
#pickle.dump((dv, model), f_out)
#f_out.close()

#better way of writing the above code is below, in that method, you  dont have to close manually

In [44]:
with open(output_file, 'wb') as f_out:
  pickle.dump((pipeline), f_out)

Load the model

In [45]:
import pickle

In [46]:
model_file = 'model_C = 1.0.bin'

In [47]:
with open(model_file, 'rb') as f_in:
  pipeline = pickle.load(f_in)

In [48]:
pipeline

Pipeline(steps=[('dictvectorizer', DictVectorizer()),
                ('logisticregression',
                 LogisticRegression(max_iter=1000, solver='liblinear'))])

In [53]:
customer = {
    "gender": "male",
    "seniorcitizen": 0,
    "partner": "no",
    "dependents": "yes",
    "phoneservice": "no",
    "multiplelines": "no_phone_service",
    "internetservice": "dsl",
    "onlinesecurity": "no",
    "onlinebackup": "yes",
    "deviceprotection": "no",
    "techsupport": "no",
    "streamingtv": "no",
    "streamingmovies": "no",
    "contract": "month-to-month",
    "paperlessbilling": "yes",
    "paymentmethod": "electronic_check",
    "tenure": 6,
    "monthlycharges": 29.85,
    "totalcharges": 129.85
}

In [54]:
pipeline.predict_proba(customer)[:, 1]

array([0.54081508])